<a href="https://colab.research.google.com/github/InesAnindiyta/BASISDATA/blob/main/sp_fc_tg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Microsoft Windows [Version 10.0.19045.5965]
(c) Microsoft Corporation. All rights reserved.

C:\Users\User>CD C:\xampp\mysql\bin

C:\xampp\mysql\bin>mysql -u root -p
Enter password:
Welcome to the MariaDB monitor.  Commands end with ; or \g.
Your MariaDB connection id is 57
Server version: 10.4.32-MariaDB mariadb.org binary distribution

Copyright (c) 2000, 2018, Oracle, MariaDB Corporation Ab and others.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

MariaDB [(none)]> create database book_store;
Query OK, 1 row affected (0.001 sec)

MariaDB [(none)]> use book_store;
Database changed
MariaDB [book_store]> create table buku (
    -> id_buku INT AUTO_INCREMENT PRIMARY KEY,
    -> judul VARCHAR(100),
    -> penulis VARCHAR(100),
    -> harga DECIMAL(10,2),
    -> stok INT
    -> );
Query OK, 0 rows affected (0.026 sec)

MariaDB [book_store]> create table pelanggan (
    -> id_pelanggan INT AUTO_INCREMENT PRIMARY KEY,
    -> nama VARCHAR(100),
    -> total_belanja DECIMAL(10,2) DEFAULT 0,
    -> status_member ENUM('REGULER','GOLD','PLATINUM') DEFAULT
    -> 'REGULER'
    -> );
Query OK, 0 rows affected (0.024 sec)

MariaDB [book_store]> create table transaksi (
    -> id_transaksi INT AUTO_INCREMENT PRIMARY KEY,
    -> id_pelanggan INT,
    -> id_buku INT,
    -> jumlah INT,
    -> total_harga DECIMAL(10,2),
    -> tanggal_transaksi DATE,
    -> FOREIGN KEY (id_pelanggan) REFERENCES
    -> pelanggan(id_pelanggan),
    -> FOREIGN KEY (id_buku) REFERENCES buku(id_buku)
    -> );
Query OK, 0 rows affected (0.052 sec)

MariaDB [book_store]> DELIMITER //
MariaDB [book_store]> CREATE FUNCTION hitung_diskon(total_belanja DECIMAL(10,2))
    -> RETURNS DECIMAL(5,2)
    -> DETERMINISTIC
    -> BEGIN
    ->  DECLARE diskon DECIMAL(5,2);
    ->  IF total_belanja >= 5000000 THEN
    ->          SET diskon = 0.10;
    ->  ELSEIF total_belanja >= 1000000 THEN
    ->          SET diskon = 0.05;
    ->  ELSE
    ->          SET diskon =0.00;
    ->  END IF;
    ->  RETURN diskon;
    -> END //
Query OK, 0 rows affected (0.010 sec)

MariaDB [book_store]> DELIMITER //
MariaDB [book_store]> CREATE PROCEDURE tambah_transaksi(
    ->          IN p_id_pelanggan INT,
    ->          IN p_id_buku INT,
    ->          IN p_jumlah INT
    -> )
    -> BEGIN
    ->          DECLARE v_harga_buku DECIMAL(10,2);
    ->          DECLARE v_total_harga DECIMAL(10,2);
    ->          DECLARE v_stok INT;
    ->          SELECT harga, stok INTO v_harga_buku, v_stok
    ->          FROM buku
    ->          WHERE id_buku = p_id_buku;
    ->          IF v_stok < p_jumlah THEN
    ->                  SIGNAL SQLSTATE '45000'
    ->                  SET MESSAGE_TEXT = 'stok buku tidak mencukupi';
    ->          END IF;
    ->          SET v_total_harga = v_harga_buku * p_jumlah;
    ->          UPDATE buku
    ->          SET stok = stok - p_jumlah
    ->          WHERE id_buku = p_id_buku;
    ->          INSERT INTO transaksi (id_pelanggan, id_buku, jumlah, total_harga, tanggal_transaksi)
    ->          VALUES (p_id_pelanggan, p_id_buku, p_jumlah, v_total_harga, CURDATE());
    ->          UPDATE pelanggan
    ->          SET total_belanja = total_belanja + v_total_harga
    ->          WHERE id_pelanggan = p_id_pelanggan;
    ->          SELECT 'transaksi berhasil' AS pesan;
    -> END //
Query OK, 0 rows affected (0.016 sec)

MariaDB [book_store]> DELIMITER;
    -> ;
    -> );
    -> );;
    -> END//
ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'DELIMITER;;
);
);;
END' at line 1
MariaDB [book_store]> CREATE TRIGGER update_status_member
    -> AFTER UPDATE ON pelanggan
    -> FOR EACH ROW
    -> BEGIN
    ->          DECLARE new_status ENUM('REGULER', 'GOLD', 'PLATINUM');
    ->          IF NEW.total_belanja >= 5000000 THEN
    ->                  SET new_status ='PLATINUM';
    ->          ELSEIF NEW.total_belanja >= 1000000 THEN
    ->                  SET new_status = 'GOLD';
    ->          ELSE
    ->                  SET new_status = 'REGULER';
    ->          END IF;
    ->          -- Hanya update jika status berbeda
    ->          IF NEW.status_member <> new_status THEN
    ->                  UPDATE pelanggan
    ->                  SET status_member = new_status
    ->                  WHERE id_pelanggan = NEW.id_pelanggan;
    ->          END IF;
    -> END //
Query OK, 0 rows affected (0.016 sec)

MariaDB [book_store]> exit
Bye

C:\xampp\mysql\bin>mysqldump -u root -p > D:\book_store.sql
Usage: mysqldump [OPTIONS] database [tables]
OR     mysqldump [OPTIONS] --databases DB1 [DB2 DB3...]
OR     mysqldump [OPTIONS] --all-databases
OR     mysqldump [OPTIONS] --system=[SYSTEMOPTIONS]]
For more options, use mysqldump --help

C:\xampp\mysql\bin>mysqldump -u root -p > D:\book_store_backup.sql
Usage: mysqldump [OPTIONS] database [tables]
OR     mysqldump [OPTIONS] --databases DB1 [DB2 DB3...]
OR     mysqldump [OPTIONS] --all-databases
OR     mysqldump [OPTIONS] --system=[SYSTEMOPTIONS]]
For more options, use mysqldump --help

C:\xampp\mysql\bin>